https://huggingface.co/docs/diffusers/v0.13.0/en/training/dreambooth


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/huggingface/diffusers

Cloning into 'diffusers'...
remote: Enumerating objects: 48263, done.
remote: Counting objects: 100% (13086/13086), done.
remote: Compressing objects: 100% (835/835), done.
remote: Total 48263 (delta 12658), reused 12344 (delta 12209), pack-reused 35177
Receiving objects: 100% (48263/48263), 31.28 MiB | 8.71 MiB/s, done.
Resolving deltas: 100% (35934/35934), done.


In [ ]:
!pip install git+https://github.com/huggingface/diffusers
!pip install -U -r diffusers/examples/dreambooth/requirements.txt

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-gdashxj3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-gdashxj3
  Resolved https://github.com/huggingface/diffusers to commit 93ea26f272f69a0e27afaebc96b68a2221a7eda0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.25.0.dev0-py3-none-any.whl size=1818207 sha256=90f8840654f7cf0d9a39764f883b2cdc9eb9103cf70fe0ee947bb577a54f176c
  Stored in directory: /tmp/pip-ephem-wheel-cache-2l1pi4po/wheels/f7/7d/99/d361489e5762e3464b3811bc629e94cf5bf5ef44dd5c3c4d52
Successfully built diffusers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 51.9 MB/s eta 0:00:

In [ ]:
!accelerate config default

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
# Training on a 16GB GPU (version that worked for training)

!pip install bitsandbytes

%env MODEL_NAME=CompVis/stable-diffusion-v1-4
%env INSTANCE_DIR = /content/gdrive/MyDrive/CV_Final_Project/Data/training_data
%env CLASS_DIR = /content/gdrive/MyDrive/CV_Final_Project/Data/class_data
%env OUTPUT_DIR = /content/gdrive/MyDrive/CV_Final_Project/Code/model

!accelerate launch /content/gdrive/MyDrive/CV_Final_Project/Code/train.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="a photo of rapunzel girl" \
  --class_prompt="a photo of princess iconic character" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=2 --gradient_checkpointing \
  --use_8bit_adam \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=20 \
  --max_train_steps=300

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
env: MODEL_NAME=CompVis/stable-diffusion-v1-4
env: INSTANCE_DIR=/content/gdrive/MyDrive/CV_Final_Project/Data/training_data
env: CLASS_DIR=/content/gdrive/MyDrive/CV_Final_Project/Data/class_data
env: OUTPUT_DIR=/content/gdrive/MyDrive/CV_Final_Project/Code/model
2023-12-15 12:33:32.189616: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 12:33:32.189720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 12:33:32.339107: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 12:33:35.1

In [ ]:
# we have separate inference.py so that everyone can locally call and generate images with CPU
# however with the below code we can generate faster in this google colab environment (due to GPU)

from diffusers import StableDiffusionPipeline
import torch

# setting fixed seed for reproducibility
seed = 123
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

model_id = "/content/gdrive/MyDrive/CV_Final_Project/Code/model"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, device_map=None).to("cuda")

result_path = "/content/gdrive/MyDrive/CV_Final_Project/Result/"

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
torch.manual_seed(seed + 21)
prompt1 = "A photo of rapunzel girl driving a car in city."
image = pipe(prompt1, num_inference_steps=50, guidance_scale=7.5).images[0]

image.save(result_path + "rapunzel1.png")

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
torch.manual_seed(seed + 999)
prompt2 = "A portrait of rapunzel girl reading a book."
image = pipe(prompt2, num_inference_steps=50, guidance_scale=7.5).images[0]

image.save(result_path + "rapunzel2.png")

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
torch.manual_seed(seed + 3333)
prompt3 = "A photo of rapunzel girl preparing for Computer Vision course with computer."
image = pipe(prompt3, num_inference_steps=50, guidance_scale=7.5).images[0]

image.save(result_path + "rapunzel3.png")

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
torch.manual_seed(seed + 15)
prompt4 = "A photo of rapunzel girl looking into mirror."
image = pipe(prompt4, num_inference_steps=50, guidance_scale=7.5).images[0]

image.save(result_path + "rapunzel4.png")

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
torch.manual_seed(seed + 757)
prompt5 = "A image of rapunzel girl speaking with phone."
image = pipe(prompt5, num_inference_steps=70, guidance_scale=7.5).images[0]

image.save(result_path + "rapunzel5.png")

  0%|          | 0/70 [00:00<?, ?it/s]